In [89]:
import numpy as np

def F(x):
    return np.array([
        3 * x[0] - np.cos(x[1] * x[2]) - 0.5, 
        x[0]**2 - 81 * (x[1] + 0.1)**2 + np.sin(x[2]) + 1.06, 
        np.exp(-x[0] * x[1]) + 20 * x[2] + (10 * np.pi - 3) / 3
    ])

x0 = np.array([0.1, 0.1, -0.1])

**Método de Newton para Sistemas Não-Lineares**

In [90]:
def J(F, x0, h = 1e-10):
    n = len(x0)
    Fx0 = F(x0)
    J = np.zeros((n, n))
    
    for i in range(n):
        x = x0
        x[i] += h
        Fx = F(x)
    
        J[:, i] = (Fx - Fx0) / h
    return J

def newton(F, J, x0, e = 1e-5, max_i = 100):
    x = x0
    for i in range(max_i):
        Fx = F(x)
        Jx = J(F,x)

        y = np.linalg.solve(Jx, -Fx)
        x = x + y
        
        if np.linalg.norm(y) < e: break
    return x

solution = newton(F, J, x0.copy())

print(solution)

[ 5.00000000e-01  1.99289049e-10 -5.23598775e-01]


**Método de Broyden**

In [91]:
def broyden(F, x0, e = 1e-5, max_i = 100):
    n = len(x0)
    x = x0
    Fx = F(x)
    
    A = np.eye(n)
    
    for i in range(max_i):
        s = np.linalg.solve(A, -Fx)
        xi = x + s
        Fxi = F(xi)
    
        y = Fxi - Fx
        A += np.outer((y - A @ s), s) / np.dot(s, s)
        
        x = xi
        Fx = Fxi
        
        if np.linalg.norm(s) < e: break
    return x


solution = broyden(F, x0)

print(solution)

[ 5.00000000e-01  8.76988305e-08 -5.23598773e-01]


**Método do Gradiente**

In [92]:
def find_alpha(F, x, z):
    alpha = 1
    g = lambda  x: 0.5 * np.linalg.norm(F(x))**2
    gi = g(x)
    
    while g(x - alpha * z) >= gi: alpha /= 2
    
    return alpha

def gradient(F, J, x0, e = 1e-5, max_i = 1000):
    x = x0
    for i in range(max_i):
        Fx = F(x)
        Jx = J(F, x)
        
        g = Jx.T @ Fx
        
        if np.linalg.norm(g) < e: break
        
        alpha = find_alpha(F, x, g)
        
        x = x - alpha * g
    return x

solution = gradient(F, J, x0)

print(solution)


[ 4.99999206e-01 -1.71600991e-08 -5.23598785e-01]


**Método da Continuação**

In [93]:
def continuation(F, x0,  max_i = 100):
    h = 1.0 / max_i
    b = -h * F(x0)
    
    x = x0

    for i in range(max_i):
        A1 = J(F, x)
        k1 = np.linalg.solve(A1, b)
        
        A2 = J(F, x + 0.5 * k1)
        k2 = np.linalg.solve(A2, b)
        
        A3 = J(F, x + 0.5 * k2)
        k3 = np.linalg.solve(A3, b)
        
        A4 = J(F, x + k3)
        k4 = np.linalg.solve(A4, b)
        
        x = x + (k1 + 2*k2 + 2*k3 + k4) / 6
        
    return x

solution = continuation(F, x0)

print(solution)


[ 0.53833727  0.47577383 -0.5187311 ]
